# Moneta Tutorial

Moneta is a tool that takes executables and records all accesses to memory as the executable runs. This record will from now on be referred to as a trace. You can create it, plot it, analyze it, and delete it. In this tutorial, you will learn the basics of using Moneta by running and tracing an example program.


## Starting Moneta

To start off, first open the link to Moneta in your preferred web browser. You should see a list of files in your browser. Then, on the top-right, choose `New > Terminal` to open a Terminal in a new tab. After this, go back to the previous tab with all the files and select `Moneta.ipynb`. A new tab will open for Moneta. You should have 3 tabs open, which we will refer to as the **Home**, **Terminal**, and **Moneta** tabs respectively.

In the **Moneta** tab, you will see a cell with the following text:
```
%run main/moneta
```
Click on the cell and either press `SHIFT + ENTER` on your keyboard, or click the `Run` button on the topmost navbar.

## Your First Trace

Now that Moneta has started up, let's run your first memory trace. Switch to the **Terminal** tab. You should be in the `~/work/moneta` directory (`cd` if you are not). This is also the directory you see in the **Home** tab. We will refer to this as the `moneta` directory.

In the `moneta` directory, you should see a folder called `tutorial`. `cd` to this folder and you should see a file called `intro.cpp` inside. You will use this program for your first trace. Open `intro.cpp` and take a look at the code (Don't worry if you don't fully understand what the code does. Just know that each `mystery` function accesses the parameter vector in a certain pattern).
```bash
cd tutorial # Alternate: cd ~/work/moneta/tutorial
vim intro.cpp
```

Throughout this tutorial, you will complete the following 3 tasks based off `intro.cpp`:
 1. Find the region in the plot containing all the vectors
 2. Find the region in the plot containing the `X`
 3. Display only the writes and read-misses of `A`, `C`, and `D`.



### Generating Your Trace

Exit the file, compile `intro.cpp` with `04`, and save the executable as `intro`.
```bash
g++ intro.cpp -04 -o intro
```

Go back to the **Moneta** tab. You will see a few customization options for generating a trace.

The **Cache Lines** and **Block Size** will change depending on your cache size. For this tutorial, you can leave these values as the defaults. 

The **Lines to Output** determines the maximum memory accesses Moneta will trace before stopping. `intro.cpp` should have fewer than 10,000,000 memory accesses, so you can leave this value as the default as well. For larger and/or longer running programs (i.e. programs with more than 10,000,000 memory accesses), you may need to increase this number so your target memory region is traced before Moneta stops. But, do note that larger values will load slower and can potentially crash Jupyter's notebook kernel.

All paths and directories in Moneta are relative to the `moneta` directory. Since `intro` is located at `moneta/tutorial/intro`, enter `tutorial` as the **Working Directory** and `intro` as the **Executable Path** 
> Side Note: Leaving **Working Directory** empty and putting `tutorial/intro` as the **Executable Path** also works since `intro` doesn't need to run specifically in the **Working Directory**

For the **Name for Trace**, you can enter any name you want. We will use the name `intro_trace` for this tutorial.

For the toggles, switch the first toggle from **Tagged trace** to **Full trace**. Leave the second toggle as is.

The inputs should be as follows:

<img src="https://raw.githubusercontent.com/NVSL/CSE141pp-Tool-Moneta/assets/TutorialFullInputs.png?token=AEXWGP7ML6FG64BE626RT6S7S3UBG" alt="Tutorial Full Inputs" width="750px"> 

Click the **Generate Trace** button and wait for the trace to finish generating.

### Exploring the Memory Access Plot

Once the trace finishes generating, you should see `intro_trace` appear under the **Full Trace** box on the right. Select the `intro_trace` and click **Load Trace**. You should see a plot appear. 

You can use the navigation features in the plot's top navbar to move around and zoom in/out of the plot. For this part, disregard all the Legend dropdowns except **Click Zoom**. If you want to use **Click Zoom**, a small zoomed graph will be displayed in the **Click Zoom** dropdown when you use the corresponding navbar tool.

For this tutorial, we want to look at the memory accesses of all the vectors, but we especially want to find the hidden `X` in the accesses. Using the navbar zoom tools, try to locate and zoom into the area where the vectors are (it will be obvious when you find this area). Spend at **most** one minute. This is just to familiarize yourself with the zoom/navigation options. If you were able to find this area, use the remaining time to find the `X`.

A bit difficult, right? All the other memory accesses are cluttering the plot!


## Tagging the Trace

Since we only care about vector memory accesses, we will need to tag the program to mark only the accesses we care about.

Moneta uses the following tagging functions to determine what, where, when, and how to trace:
```c++
DUMP_START_SINGLE(const char* tag, const void* begin, const void* end)
DUMP_START_MULTI(const char* tag, const void* begin, const void* end)
DUMP_STOP(const char* tag)
FLUSH_CACHE()
```

To use the functions, you will need to `#include` the `pin_tags.h` (this is done for you in `intro.cpp`). This file is in the `moneta` directory:
```c++
#include "../pin_tags.h" // intro.cpp is in "tutorial", ".." brings you to "moneta"
```

You can use `DUMP_START_SINGLE()` or `DUMP_START_MULTI()` to specify an address range to trace. Since we only need to trace the vector regions once, we will use `DUMP_START_SINGLE()` here. For `DUMP_START_SINGLE()`, you will need to include a tag name/label, and the beginning and ending addresses of the memory range to trace. 

For this tutorial, we will use the vectors' variable names as tag names: `A`, `B`, etc. Since vectors are contiguous, the start and end memory addresses are simply the addresses of the first and last elements of the vector: `&vector.front()` and `&vector.back()` respectively. `&vector[0]` and `&vector[SIZE-1]` works as well.

Go to the **Terminal** tab and open `intro.cpp`. In `main`, surround mystery functions `A` to `H` with a `DUMP_START_SINGLE()` call and a corresponding `DUMP_STOP()` call with the same tag. The first few have been done for you in `intro.cpp`:
```c++
DUMP_START_SINGLE("A", &A.front(), &A.back())
mystery_a(A)
DUMP_STOP("A")

DUMP_START_SINGLE("B", &B.front(), &B.back())
mystery_b(B)
DUMP_STOP("B")
//And so on...
```

Save and exit, compile `intro.cpp` with `04`, and save the executable as `intro_tagged`.
```bash
g++ intro.cpp -04 -o intro_tagged
```

Return to the **Moneta** tab and generate a new trace with the following options. Make sure to toggle from **Full trace** to **Tagged trace**:

<img src="https://raw.githubusercontent.com/NVSL/CSE141pp-Tool-Moneta/assets/TutorialTaggedInputs.png?token=AEXWGPYH35VMWOLSZMSQ26C7S3UDE" alt="Tutorial Tagged Inputs" width="750px"> 

Click the "Generate Trace" button and wait for the trace to finish generating.

## Exploring the Memory Access Plot, But Better This Time

Once generated, you should see `intro_tagged_trace` appear under the **Tagged Trace** box on the right. Select and load this trace. There's a lot fewer data points cluttering our vectors!

### Finding the Vector Region

As you can see from the plot, you are already zoomed into the vector region and already finished the first task for finding the vectors. The tagging did all the work for you!

### X Marks the Spot

Now, you need to find the `X` access pattern hidden in one of the vectors. Since you tagged memory regions of interest, you can zoom directly to these regions through the **Tags** dropdown in the Legend. Open **Tags** and zoom to the various vectors using the zoom button next to each tag until you find the `X` and complete the second goal.

### Display Specific Accesses

For your final goal, you need to display only the writes and read-hits of `A`, `C` and `D`. For this task, you will need the **Accesses** and **Tags** section of the Legend.

First, zoom back out so that you can see all the vectors again. This can be done using the **Reset Zoom** button in the navbar.

In the **Tags**, you may have already noticed the checkboxes. These checkboxes allow you to choose whether the tag is displayed on the plot or not. Using these checkboxes, make it so that the plot only displays vectors/tags `A`, `C` and `D`. A few notes:
 * You should not need to touch `Stack` or `Heap` for this
 * For tags with overlapping address ranges, disabled checkboxes take priority (there should be no overlapping ranges in this tutorial)

In the **Accesses** section, you also have the ability to show/hide certain access types. Using these checkboxes, display all writes, but only the read-hits to finish the last task. You can find out what each icon means by hovering over their corresponding checkbox.

### Solutions
 - The hidden `X` is in vector `B`.
 - In **Tags**, everything should be unchecked except `Stack`, `Heap`, `A`, `C` and `D`.
 - In **Accesses**, everything in the **Writes** columns should be checked, and only the checkbox intersecting **Reads** and **Hits** should be checked.



# Lab
***

# Analyzing Cache Hit Rate

In this lab, you will be analyzing and eventually optimizing `lab.cpp`, a transpose sum operation, using Moneta to guide you.

## Tasks to Perform



### Inspect the Code

You need to get acquainted with the code in order to optimize it.

There are two 128x128 2D arrays, `A` and `B`. The entire operation can be summarized by 
> sum(A + B.T)

where `sum()` adds up all the values of the matrix.

We will use Moneta to gather the answers.

### Constraints
- Unless otherwise stated, use a cache with 128 lines and a 64-byte block size.
- Compile the program with `-O4`
- Make sure to get the rates for just the accesses to `A` and `B`
- `FLUSH_CACHE` is not needed and using it could result in far worse hit rates

### Questions

1. What is the baseline hit rate for the entire operation (the outermost loop)?
2. Does doubling the block size affect the miss rate? Why/why not?
3. Does doubling the number of cache lines affect the miss rate? Why/why not?
4. How many cache lines are used by the array `A` during one iteration of the row loop?
5. What about `B`?
6. How would you modify the code to minimize the total miss rate?

## Tips and Guidelines
Several features of Moneta can come in handy while completing this lab.  

### Tagging
Initially, adding tags to the source code can help orient the user by quickly bringing up the locations of the arrays in the traces.

**DUMP_START_SINGLE** - This function can be useful to tag where the entire array such as `A` is located.  
> **Note:** The first element of `A` has the address `A` and the last element is pointed to by `A + N * N - 1`

**DUMP_START_MULTI** - Separate calls to this function create new tags making locating specific iterations of nested loops straightforward.
> **Note:** Tags are 0-indexed so `DUMP_START_MULTI("loop", ...)` results in `loop0, loop1, ...`

### Analyzing
The UI contains features to access these tags.

**Tags** - The Tags panel contains any tags Moneta picks up. Each tag allows you to zoom in to the region and on hover displays region stats

**Stats** - Current stats in the Stats panel can display the rates of the entire operation if both arrays appear in the plot.

### Other
- Feel free to create lots of traces with meaningful names to keep track of your modifications
- If tags haven't been added or don't work, start with a full trace and track from main to find the arrays manually
- When changing cache parameters, think about how much space the arrays take (`int`s are 4 bytes)
- Recompile the program whenever you make changes

